In [1]:
import pandas as pd

In [2]:
pd.set_option("display.max_rows", 300)  #表示する行数を増やす

In [3]:
#明智は人名、大湫はツイートなし、日吉は京都にも同じ地名あり、山岡は人名ラーメン屋、釜戸はご飯、明世はツイートなし、土岐は人名、陶は人名器
city_list = ['akechi', 'daishuu', 'ena', 'hiyoshi', 'inetsu', 'iwamura', 'kamako', 'kushihara',
             'meiyo', 'mizunami', 'nakatsugawa', 'tajimi', 'toki', 'tou', 'ueyahagi', 'yamaoka']
city = city_list[1]
period = '2018-06-30'
infile = "tweets_in_a_week/"+city+period+".csv"
outfile = "tweets_in_a_week/"+city+period+".png"

In [4]:
datas = pd.read_csv(open(infile, 'rU'), encoding='utf-8')

/Users/a0082273/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 'U' mode is deprecated
  """Entry point for launching an IPython kernel.


In [5]:
datas

,Unnamed: 0,time,id,name,text
0,0,Fri Jul 06 23:24:37 +0000 2018,r_21_gifu,岐阜県交通情報(非公式),※解除※\n恵那御嵩線(６５号線) 下り 区間：瑞浪市釜戸町－瑞浪市大湫町\n大雨のため通行...
1,1,Fri Jul 06 23:24:35 +0000 2018,r_21_gifu,岐阜県交通情報(非公式),※解除※\n恵那御嵩線(６５号線) 上り 区間：瑞浪市大湫町－瑞浪市釜戸町\n大雨のため通行...
2,2,Fri Jul 06 03:22:31 +0000 2018,r_21_gifu,岐阜県交通情報(非公式),恵那御嵩線(６５号線) 下り 区間：瑞浪市釜戸町－瑞浪市大湫町\n大雨のため通行止 (7/6...
3,3,Fri Jul 06 03:22:28 +0000 2018,r_21_gifu,岐阜県交通情報(非公式),恵那御嵩線(６５号線) 上り 区間：瑞浪市大湫町－瑞浪市釜戸町\n大雨のため通行止 (7/6...


In [6]:
datas = datas.drop('Unnamed: 0', axis=1)

In [7]:
for col in range(datas.shape[0]):
    if type(datas.name[col]) != str or type(datas.text[col]) != str:
        print(col, datas.iloc[col])
        datas = datas.drop(col, axis=0)
datas = datas.reset_index(drop=True)

In [8]:
improper_names  = ['bot', 'ぼっと', '情報', '案内', '相互', '出会', 'セフレ', 'エッチ', '法人',
                   'フィギュア', 'リトルアーモリー', '空き家オールバンク', 'つけめん 恵那く',
                   'えな', '宮部雪宗', '多治見ネオ']
improper_texts  = ['improper_names', '岩村明憲', '岩村努', '岩村愛', "I'm at", "稲津けんご", '相互', '吉田明世']

for col in range(datas.shape[0]):
    for word in improper_names:
        if word in datas['name'][col].lower():
            datas['text'][col] = 'improper_names'
            break
    for word in improper_texts:
        if word in datas['text'][col].lower():
            datas = datas.drop(col, axis=0)
            break

datas = datas.reset_index(drop=True)

In [9]:
datas

,time,id,name,text


In [10]:
import MeCab
import re

m = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")
word_list = []
for i in range(datas.shape[0]):
    if type(datas.text[i]) == str:
        texts = m.parse(datas['text'][i])
        texts = texts.split('\n')
        for text in texts:
            text = re.split('[\t,]', text)
            if text[0] == 'EOS' or text[0] == '':
                pass
            elif text[1] == '名詞' and text[2] == '一般': 
                word_list.append(text[0])
            elif text[1] == '名詞' and text[2] == 'サ変接続': 
                word_list.append(text[0])
            elif text[1] == '名詞' and text[2] == '形容動詞語幹': 
                word_list.append(text[0])
            elif text[1] == '形容詞' and text[2] == '自立': 
                word_list.append(text[0])

In [11]:
from collections import Counter

counter = Counter(word_list)
for word, cnt in counter.most_common():
    print(word, cnt)

In [12]:
stop_words = ['in', 'ー', 'bot', 'ない', '無い']

def remove_specified_values(arr, value):
    while value in arr:
        arr.remove(value)

for word in stop_words:
    remove_specified_values(word_list, word)

In [13]:
counter = Counter(word_list)
for word, cnt in counter.most_common():
    print(word, cnt)

In [14]:
word_list = ' '.join(word_list)

In [15]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

fpath = "~/Library/Fonts/RictyDiminished-Regular.ttf"

wordcloud = WordCloud(background_color="white", font_path=fpath, width=900, height=500, max_words=80).generate(word_list)

plt.figure(figsize=(10,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig(outfile)
plt.show()
#一文字の単語は表示してくれない...

ValueError: We need at least 1 word to plot a word cloud, got 0.